# Imports

In [1]:
import pandas as pd

# Loads

In [2]:
df_bike = pd.read_csv('Material/nyc_bicycle.csv')
df_stations = pd.read_csv('Material/nyc_stations.csv')
df_weather = pd.read_csv('Material/nyc_weather.csv')
df_traffic = pd.read_csv('Material/nyc_traffic.csv')
mighty_df = pd.read_csv('Material/data.csv')

# Preprocessing nyc count data

In [9]:
# Add long lat from df_stations 
counts_with_all = df_bike.merge(df_stations, how='outer', on='site')

# Take only valuable columns
counts_with_all = counts_with_all[['longitude','latitude', 'date', 'counts']]

# add several columns for time and change the time column type to satetime
counts_with_all['time']  = pd.to_datetime(counts_with_all['date'], format='%m/%d/%Y %I:%M:%S %p')
counts_with_all['hour'] = counts_with_all['time'].dt.hour
counts_with_all['min'] = counts_with_all['time'].dt.minute

# Reduce granularity of the observations
counts_with_all = counts_with_all[counts_with_all['min']==0]

# select necessary columns
counts_with_all = counts_with_all[['longitude','latitude', 'time','hour', 'counts']]
counts_with_all.columns = ['longitude','latitude', 'time','hour', 'count']

# Indicate that those columns are for bucycle data
counts_with_all.insert(4, 'type', 'bicycle')
df_weather['time'] =  pd.to_datetime(df_weather['time'])

# Merge nyc bike counts and weatherr
bikes = df_weather.merge(counts_with_all, how='inner', on='time')

# Indicate the city the columns are from
bikes.insert(4, 'city', 'New-York')

#pre processing vehicle data
df_vh_nyc = df_traffic.copy()
df_vh_nyc['Date'] = pd.to_datetime(df_vh_nyc['Date'], format='%Y-%m-%dT%H:%M:%S.%f')
df_vh_nyc = df_vh_nyc.drop(['Roadway Name', 'From', 'To', 'Direction'], axis=1)

# rename the columns 
df_vh_nyc.columns = [  'ID','Segment ID', 'date', '0', '1',
                       '2', '3', '4', '5',
                       '6', '7', '8', '9',
                       '10', '11', '12', '13',
                       '14', '15', '16', '17',
                       '18', '19', '20', '21',
                       '22', '23']
df_vh_nyc['date'] = pd.to_datetime(df_vh_nyc['date'])

# Put hourly count columns in rows of 1 column with name hour. Save all the values into count
df_vh_nyc_melt = df_vh_nyc.melt(id_vars = ['date','ID','Segment ID'],
                         var_name = 'hour', 
                         value_name = "count").sort_values('date')

# We dont have coordinate data for these columns, so we fill them with -9999
df_vh_nyc_melt.insert(4, 'latitude', -9999)
df_vh_nyc_melt.insert(4, 'longitude', -9999)

# Rename columns to fit general format
df_vh_nyc_melt.columns = ['time', 'ID', 'Segment ID', 'hour', 'longitude', 'latitude', 'count']

# Add weather data
df_nyc_traf = df_weather.merge(df_vh_nyc_melt, how='inner', on='time')

# Add city column
df_nyc_traf.insert(4, 'city', 'New-York')

# Add transportation type column
df_nyc_traf.insert(4, 'type', 'vehicle')

# Select only useful columns in the right order to fit general df
df_nyc_traf = df_nyc_traf[['time', 'temp', 'dwpt', 'rhum', 'city', 'wdir', 'wspd', 'pres',
       'longitude', 'latitude', 'hour', 'type', 'count', 'ID', 'Segment ID']]

# cancatinate bike and traffic data
nyc = pd.concat([df_nyc_traf,bikes], axis=0)

# Save it into the intermediate file
nyc.to_csv('Material/nyc.csv')

# Concatinate nyc data to all the other cities

In [ ]:
# Change order of columns in the general database
mighty_df= mighty_df[['time', 'temp', 'dwpt', 'rhum', 'city', 'wdir', 'wspd', 'pres',
       'longitude', 'latitude', 'hour', 'type', 'count']]

# add New-york data to all data
all_df = pd.concat([mighty_df,nyc], axis=0)

# Reset index and drop the column
all_df = all_df.reset_index()
all_df= all_df.drop('index', axis=1)

# Save it in the file
all_df.to_csv('Material/nyc_cop_lon.csv')

In [ ]:
all_df